In [1]:
import cv2
import pandas as pd
import numpy as np
import os
from skimage.feature.texture import greycomatrix
from skimage.feature.texture import greycoprops
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from tqdm import tqdm

In [7]:
def get_feature(img):
    
    img_graymatrix = greycomatrix(img, [1], [0, np.pi/2])
#     print("img_graymatrix shape: ",img_graymatrix.shape)
    
    img_contrast = greycoprops(img_graymatrix, 'contrast')
#     print("img_contrast shape: ", img_contrast.shape)
    
    img_homogeneity = greycoprops(img_graymatrix, 'homogeneity')
#     print("img_homogeneity shape: ", img_homogeneity.shape)
    
    img_correlation = greycoprops(img_graymatrix, 'correlation')
#     print("img_correlation shape: ", img_correlation.shape)
    
    
    img_contrast_flattened = img_contrast.flatten()
    img_homogeneity_flattened = img_homogeneity.flatten()
    img_correlation_flattened = img_correlation.flatten()
    
    features = np.concatenate([img_contrast_flattened, img_homogeneity_flattened, 
                             img_correlation_flattened])
#     print("final_feature shape: ",features.shape)
    
    return(features)

In [8]:
def feature_extraction(path_to_folder, class_label):
    data_list=[]
#     count=1
    for file_name in tqdm(os.listdir(path_to_folder)):
#         if(count>1):
#             break
        path_to_img = os.path.join(path_to_folder,file_name)
        img = cv2.imread(path_to_img, 0) # grayscale image
         
        if np.shape(img) == ():
            continue
        
        final_feature = get_feature(img)
#         print("final_feature shape: ",final_feature.shape)
#         print("final_feature is: ",final_feature)
        
        final_feature=list(final_feature)
        final_feature.insert(0,file_name)
        final_feature.insert(1,class_label)
        
        data_list.append(final_feature)
        
#         count+=1
       
    return(data_list)

In [9]:
nonfog_path = 'path_to_folder_having_NON-FOGGY_images'
fog_path = 'path_to_folder_having_FOGGY_images'

data_list1 = feature_extraction(nonfog_path, 0)
data_list2 = feature_extraction(fog_path, 1)

df = pd.DataFrame(data_list1)
df = df.append(pd.DataFrame(data_list2), ignore_index=True)
# #  --------------------------------------------------------------------------------------


df.rename(columns = {0: "image_names", 1: "label"}, inplace = True)


100%|██████████| 85/85 [00:01<00:00, 51.71it/s]


In [10]:
df.head()

,image_names,label,2,3,4,5,6,7
0,LIma-000042.png,0,173.710166,243.492739,0.444013,0.375746,0.981380,0.973850
1,LIma-000017.png,0,58.431123,98.475688,0.592629,0.533884,0.995635,0.992649
2,LIma-000053.png,0,181.712516,141.194132,0.515167,0.468189,0.979226,0.983835
3,LIma-0000041.png,0,573.088980,562.578777,0.398145,0.351479,0.934578,0.935729
4,LIma-000028.png,0,223.987940,307.145521,0.437860,0.385056,0.981208,0.974214


In [11]:
df.shape

(169, 8)

In [12]:
df.to_csv('features.csv', index=False)

### SVM classification

In [ ]:
# df=pd.read_csv('features.csv')

In [13]:
array=df.values
x_feature=array[:,2:]
y_label=array[:,1].astype('int')
print(x_feature.shape)
print(y_label.shape)

(169, 6)
(169,)


In [17]:
X_train,X_test,Y_train,Y_test=train_test_split(x_feature,y_label,test_size=0.10,random_state=7)

In [18]:
# Normalise the data after splitting to avoid information leak between train and test set.

scaler_norm = MinMaxScaler()

X_train = scaler_norm.fit_transform(X_train)
X_test = scaler_norm.fit_transform(X_test)

In [19]:
# Random check
model_SVC=SVC(kernel='rbf',C=100,gamma=0.001)

kfold=KFold(n_splits=10, shuffle=True)
cv_results=cross_val_score(model_SVC,X_train,Y_train,cv=kfold,scoring='accuracy')
msg="%s %f (%f)" % ('Training Accuracy: ',cv_results.mean(),cv_results.std())
print(msg)

Training Accuracy:  0.835417 (0.096668)


In [20]:
# SVM hypertuning using GridSeachCV
model_SVC=SVC()

kfold=KFold(n_splits=10)
param_grid = {'C': [1, 10, 100, 500, 1000], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']} 

grid=GridSearchCV(estimator=model_SVC,param_grid=param_grid,scoring='accuracy',cv=kfold,verbose=3)
grid_result=grid.fit(X_train,Y_train)

print("Best: %f using %s" % (grid_result.best_score_,grid_result.best_params_))

Fitting 10 folds for each of 25 candidates, totalling 250 fits
[CV] C=1, gamma=1, kernel=rbf ........................................
[CV] ............ C=1, gamma=1, kernel=rbf, score=0.688, total=   0.0s
[CV] C=1, gamma=1, kernel=rbf ........................................
[CV] ............ C=1, gamma=1, kernel=rbf, score=0.875, total=   0.0s
[CV] C=1, gamma=1, kernel=rbf ........................................
[CV] ............ C=1, gamma=1, kernel=rbf, score=1.000, total=   0.0s
[CV] C=1, gamma=1, kernel=rbf ........................................
[CV] ............ C=1, gamma=1, kernel=rbf, score=1.000, total=   0.0s
[CV] C=1, gamma=1, kernel=rbf ........................................
[CV] ............ C=1, gamma=1, kernel=rbf, score=0.867, total=   0.0s
[CV] C=1, gamma=1, kernel=rbf ........................................
[CV] ............ C=1, gamma=1, kernel=rbf, score=0.867, total=   0.0s
[CV] C=1, gamma=1, kernel=rbf ........................................
[CV] .........

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


[CV] ......... C=10, gamma=0.1, kernel=rbf, score=0.933, total=   0.0s
[CV] C=10, gamma=0.1, kernel=rbf .....................................
[CV] ......... C=10, gamma=0.1, kernel=rbf, score=0.733, total=   0.0s
[CV] C=10, gamma=0.01, kernel=rbf ....................................
[CV] ........ C=10, gamma=0.01, kernel=rbf, score=0.688, total=   0.0s
[CV] C=10, gamma=0.01, kernel=rbf ....................................
[CV] ........ C=10, gamma=0.01, kernel=rbf, score=0.812, total=   0.0s
[CV] C=10, gamma=0.01, kernel=rbf ....................................
[CV] ........ C=10, gamma=0.01, kernel=rbf, score=0.933, total=   0.0s
[CV] C=10, gamma=0.01, kernel=rbf ....................................
[CV] ........ C=10, gamma=0.01, kernel=rbf, score=0.933, total=   0.0s
[CV] C=10, gamma=0.01, kernel=rbf ....................................
[CV] ........ C=10, gamma=0.01, kernel=rbf, score=0.867, total=   0.0s
[CV] C=10, gamma=0.01, kernel=rbf ....................................
[CV] .

[CV] ..... C=100, gamma=0.0001, kernel=rbf, score=0.867, total=   0.0s
[CV] C=100, gamma=0.0001, kernel=rbf .................................
[CV] ..... C=100, gamma=0.0001, kernel=rbf, score=0.867, total=   0.0s
[CV] C=100, gamma=0.0001, kernel=rbf .................................
[CV] ..... C=100, gamma=0.0001, kernel=rbf, score=0.467, total=   0.0s
[CV] C=100, gamma=0.0001, kernel=rbf .................................
[CV] ..... C=100, gamma=0.0001, kernel=rbf, score=0.600, total=   0.0s
[CV] C=100, gamma=0.0001, kernel=rbf .................................
[CV] ..... C=100, gamma=0.0001, kernel=rbf, score=0.600, total=   0.0s
[CV] C=100, gamma=0.0001, kernel=rbf .................................
[CV] ..... C=100, gamma=0.0001, kernel=rbf, score=0.933, total=   0.0s
[CV] C=100, gamma=0.0001, kernel=rbf .................................
[CV] ..... C=100, gamma=0.0001, kernel=rbf, score=0.533, total=   0.0s
[CV] C=100, gamma=0.0001, kernel=rbf .................................
[CV] .

[CV] ...... C=1000, gamma=0.01, kernel=rbf, score=1.000, total=   0.0s
[CV] C=1000, gamma=0.01, kernel=rbf ..................................
[CV] ...... C=1000, gamma=0.01, kernel=rbf, score=1.000, total=   0.0s
[CV] C=1000, gamma=0.01, kernel=rbf ..................................
[CV] ...... C=1000, gamma=0.01, kernel=rbf, score=0.933, total=   0.0s
[CV] C=1000, gamma=0.01, kernel=rbf ..................................
[CV] ...... C=1000, gamma=0.01, kernel=rbf, score=0.867, total=   0.0s
[CV] C=1000, gamma=0.01, kernel=rbf ..................................
[CV] ...... C=1000, gamma=0.01, kernel=rbf, score=0.933, total=   0.0s
[CV] C=1000, gamma=0.01, kernel=rbf ..................................
[CV] ...... C=1000, gamma=0.01, kernel=rbf, score=1.000, total=   0.0s
[CV] C=1000, gamma=0.01, kernel=rbf ..................................
[CV] ...... C=1000, gamma=0.01, kernel=rbf, score=1.000, total=   0.0s
[CV] C=1000, gamma=0.01, kernel=rbf ..................................
[CV] .

[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed:    0.7s finished


In [22]:
model_SVC = SVC(C=500,gamma=0.1, kernel='rbf')
model_SVC.fit(X_train,Y_train) 

predictions=model_SVC.predict(X_test)

print(accuracy_score(Y_test,predictions))
print(confusion_matrix(Y_test,predictions))
print(classification_report(Y_test,predictions))

0.8235294117647058
[[6 0]
 [3 8]]
              precision    recall  f1-score   support

           0       0.67      1.00      0.80         6
           1       1.00      0.73      0.84        11

    accuracy                           0.82        17
   macro avg       0.83      0.86      0.82        17
weighted avg       0.88      0.82      0.83        17



### Conclusion:
The final result were- train-set acc. = 96.12% and test-set acc. = 82.35%.
In this implementation i have used only the synthetic dataset - http://perso.lcpc.fr/tarel.jean-philippe/bdd/frida.html called FRIDA and FRIDA0-2.
The original paper mentions 156-foggy and 156-non foggy images, but here i could only find 84-non foggy images. Thus I made a balanced dataset of just 169 total images taken from both FRIDA and FRIDA-2 datasets. Thus, the results show less accuracy due to usage of only half of the dataset. However, the feature extraction technique is proper.

Please cite this paper if you use this code for any work -
**Asery, R., Sunkaria, R. K., Sharma, L. D., & Kumar, A. (2016). Fog detection using GLCM based features and SVM. 2016 Conference on Advances in Signal Processing (CASP). doi:10.1109/casp.2016.7746140**
